## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

Now it's possible to add ec2-rds connection automaticly in UI

In [6]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "default" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "16.16.27.89" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://16.16.27.89:5000'


In [8]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-13/1', creation_time=1722011022598, experiment_id='1', last_update_time=1722011022598, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-13/0', creation_time=1722010151833, experiment_id='0', last_update_time=1722010151833, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 's3://mlflow-artifacts-remote-13/1/af2eeaa08ad3461487bf8af2f457927b/artifacts'


In [10]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-13/1', creation_time=1722011022598, experiment_id='1', last_update_time=1722011022598, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-13/0', creation_time=1722010151833, experiment_id='0', last_update_time=1722010151833, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [11]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [13]:
# client.list_registered_models()
# quod vide 2.5
client.search_registered_models()

[]

In [15]:
# run_id = client.list_run_infos(experiment_id='1')[0].run_id
# quod vide 2.5
run_id = client.search_runs('1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/07/26 18:41:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1722012061758, current_stage='None', description='', last_updated_timestamp=1722012061758, name='iris-classifier', run_id='af2eeaa08ad3461487bf8af2f457927b', run_link='', source='s3://mlflow-artifacts-remote-13/1/af2eeaa08ad3461487bf8af2f457927b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>